# Ezega Data Analysis

Author: [Meheret Samuel](https://github.com/senadev42)

<hr>

### The Pre-amble
A few months ago a friend of mine, [Kidus](https://github.com/kidesleo),  decided to scrape ezega.com, which is a sort of index for bussinesses in ethiopia. The result was a jsonl file with about 8k entries that I've been sitting on with the vague goal of looking into but not finding the time to.

Finally have time now, and while doing data analysis with clean datasets taken off the internet has been fun, it's not time to work with something dirty and real.


## Setting Up

First let's import the libraries we're going to be using.

In [ ]:
import pandas as pd

And then the file itself, setting lines=True to indicate that newlines are being used as a delimiter here because this is jsonl.

In [2]:
df = pd.read_json('ezega_data.jsonl', lines=True)

Now let's take a peek at our data set.

In [6]:
df.head()

,business_title,business_image,business_location,business_url,business_description,business_numbers,ezega_url,category,sub_category
0,Bagel Corner Bakery,https://businessguide.ezega.com/images/noimage...,Addis Ababa,None,None,"[0116513067, 0911455510, +251116352780, 011550...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Resources
1,AWASH BAR & RESTURANT,https://businessguide.ezega.com/images/noimage...,Mekelle,None,None,"[0344402815, 0344418248, 0344415657]",https://businessguide.ezega.com/Default.aspx?a...,Auto,Resources
2,BITMAS General Trading PLC,https://businessguide.ezega.com/images/noimage...,"Ras Desta Damtwe St. NTO Building, 3rd floor, ...",None,None,"[+251115501364, +251911516445+2510911133182, +...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Water Services
3,ROB VIDEO,https://businessguide.ezega.com/images/noimage...,Mekelle,None,None,[0344400112],https://businessguide.ezega.com/Default.aspx?a...,Auto,Water Services
4,Tesfa PLWHA Association,https://businessguide.ezega.com/images/noimage...,Addis Ababa,None,None,"[0467730206, 0116558830, 2511613426, 011554039...",https://businessguide.ezega.com/Default.aspx?a...,Auto,Shipping Companies


As expected, it's a bit messy. 

At a glance, 
- `business_url` and `business_description` look like they have a lot of values missing, 
- the bussiness titles are inconsistently capitalized,
- some of the locations are cities while some of them are full addresses
- and the bussiness numbers are arrays that seem to be a mix of the Country Code +251 suffix and the simple, local 0 suffix.

And there's probably other stuff that isn't immediately apparent from this quick look through, but let's work on cleaning this up first.